In [ ]:
!apt-get update
!apt-get install -y wget unzip curl
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb || apt-get -fy install
!google-chrome --version
!pip install selenium
!pip install webdriver-manager

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [5,340 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [80.2 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,297 kB]
Get:13 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jamm

### import Librarys


In [ ]:
import json
import re
import time
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from openai import OpenAI
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urljoin
from openai import OpenAI

### Here we are collecting the links for one year and getting data for every day

In [ ]:


# Extract date from URL in DD/MM/YYYY format
def get_date(url):
    match = re.search(r'www\.tribuneindia\.com/(\d{4})/(\d{4,8})', url)
    if not match:
        return ""
    year = match.group(1)
    md = match.group(2)

    if len(md) == 4:  # MMDD
        month, day = md[:2], md[2:]
    elif len(md) == 8:  # YYYYMMDD
        year, month, day = md[:4], md[4:6], md[6:]
    else:
        month, day = "01", "01"

    return f"{day}/{month}/{year}"


# Extract all valid daily article URLs from a year page
def get_links_for_one_year(year_page):
    resp = requests.get(year_page)
    soup = BeautifulSoup(resp.text, "html.parser")
    pattern = re.compile(r'^http://www\.tribuneindia\.com/\d{4}/\d{8}$')
    openable_links = set()

    for a_tag in soup.find_all("a", href=True):
        href = a_tag['href']
        if pattern.match(href):
            openable_links.add(href)
    return openable_links


# Extract headlines and article content from a single day page
def get_content_for_one_day(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "lxml-xml")

    # Base URL to resolve relative main*.htm links
    base_url = urljoin(url, '.')
    pattern = re.compile(r"^main\d+\.htm$")
    anchors = {urljoin(base_url, a['href']) for a in soup.find_all("a", href=pattern)}

    date = get_date(url)
    contents = []

    for anchor in anchors:
        r2 = requests.get(anchor)  # Use requests instead of Selenium
        article_soup = BeautifulSoup(r2.text, "html.parser")

        article_td = article_soup.find("td", {"valign": "top", "colspan": "2", "width": "569"})
        if article_td:
            # Headline
            headline_tag = article_td.find("font", {"size": "4"})
            headline = headline_tag.get_text(strip=True) if headline_tag else ""

            # Article content
            article_text = []
            font_tags = article_td.find_all("font", {"size": "2"})
            for idx, font_tag in enumerate(font_tags):
                # Combine all text in this font tag
                text = " ".join(font_tag.stripped_strings)

                # Skip the first tag if it looks like a byline / source info
                if idx == 0 and any(x for x in text.split() if "Tribune" in x or "/" in x):
                    continue

                # Append valid text
                if text:
                    article_text.append(text)

            # Join all parts and clean up whitespace
            raw_text = " ".join(article_text)
            article = re.sub(r'\s+', ' ', raw_text).strip()
            contents.append([headline, article, date, news])

    return contents


## Calling the openai to get the summary andd category of the content

In [ ]:
my_api_key_perplexity='Enter the api key'
url = "https://api.perplexity.ai/chat/completions"
headers = {
    "Authorization": f"Bearer {my_api_key_perplexity}",
    "Content-Type": "application/json"
}
def analyze_text_perplexity(content: str):
    """
    Takes a paragraph and returns a summary and category.
    """
    summary_payload = {
    "model": "sonar",
    "messages": [
        {
            "role": "system",
            "content": (
                "You are a helpful assistant designed to summarize articles into about 80-100 words. "
                "Respond with **only the summary text**, do not include JSON, explanations, or any extra formatting."
            )
        },
        {
            "role": "user",
            "content": f"Please summarize the following article in about 80-100 words: {content}"
        }
    ]
    }
    category_payload = {
        "model": "sonar",
        "messages" : [
        {
            "role": "system",
            "content": "You are a helpful assistant.Health and Wellne When given an article, respond **only** with the exact single category it belongs to from this list: Politics, International News, National News, Local News, Business and Finance, Science and Technology, Entertainment, Sports, Lifestyle and Features, Opinion and Editorial, Environment, Education, Crime and Justice, Human Interest, Obituaries, Weather, Religion and Spirituality, Technology and Gadgets, Automotive. Do not provide explanations, reasoning, or any extra text."
        },
        {
            "role": "user",
            "content": content  # your article text variable
        }
    ]
    }
    category = requests.post(url, headers=headers, json=category_payload).json()["choices"][0]["message"]["content"]
    summary = requests.post(url, headers=headers, json=summary_payload).json()["choices"][0]["message"]["content"]
    return {"summary": summary, "category": category}

In [ ]:
from openai import OpenAI
client = OpenAI(api_key="Enter the api key")

def analyze_text_openai(text: str):
    """
    Takes a paragraph and returns a summary and category.
    """

    # Get Summary
    summary_response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that summarizes text in about 80-100 words."},
            {"role": "user", "content": f"Summarize this text :\n\n{text}"}
        ]
    )
    summary = summary_response.choices[0].message.content

    # Get Category
    category_response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a classifier that assigns a category to the given text."},
            {"role": "user", "content": f"Classify this text into a category (e.g.,  Politics, International News, National News, Local News, Business and Finance, Science and Technology, Entertainment, Sports, Lifestyle and Features, Opinion and Editorial, Environment, Education, Crime and Justice, Human Interest, Obituaries, Weather, Religion and Spirituality, Technology and Gadgets, Automotive. Do not provide explanations, reasoning, or any extra text.):\n\n{text}"}
        ]
    )
    category = category_response.choices[0].message.content

    return {"summary": summary, "category": category}

## this code for getting the permission for scrape the data

In [ ]:
# Set up Chrome options and driver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('-incognito')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=chrome_options
)

#changeable information
date= '07/12/2011'
newspaper= 'The Tribune'
url = "https://www.tribuneindia.com/2003/20030105/index.htm"

# Send a GET request to the URL

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
}

r = requests.get(url, headers=headers)

print(r.status_code)  # should be 200 now

200


### getting all the links for one year

In [ ]:
year_archive_link='https://www.tribuneindia.com/2014/forms/arc03.htm'
full_year_news_links=get_links_for_one_year(year_archive_link)
print(len(full_year_news_links))

362


## Scrape all the news for all the days

In [ ]:
import pandas as pd

full_year_contents = []
count = 1
i = 1

for news in full_year_news_links:
    if count <= 1:   # remove this condition if you want all links
        day_content = get_content_for_one_day(f'{news}/index.htm')
        full_year_contents.extend(day_content)   # add all news of that day
    else:
        break
    print(i)
    i += 1

print(len(full_year_contents))

# 🔹 Convert list of dicts into DataFrame
df = pd.DataFrame(full_year_contents)

# 🔹 Save to CSV
df.to_csv("news_data.csv", index=False, encoding="utf-8-sig")
print("✅ Data saved to news_data.csv")


1
2


KeyboardInterrupt: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
suset = df.iloc[41:81]


,headline,date,article,summary,category,url


## Calling Open AI to get summary and category


In [ ]:

import math

google_form_filling_content = []
news_count = 1

for index, row in full_year_contents.iterrows():
    headline = row[0]
    text = row[1]
    date = row[2]
    url = row[3]


    # Check that all required fields are non-empty and not NaN
    if isinstance(headline, str) and headline!='' and isinstance(text, str) and text!='' and isinstance(date, str) and date!='' and isinstance(url, str) and url!='':
        summary_and_category = analyze_text_openai(text)
        summary = summary_and_category.get("summary", "")
        category = summary_and_category.get("category", "")
        # Only append if summary and category are non-empty
        if summary!='' and category!='':
            google_form_filling_content.append({
                "headline": headline,
                "date": date,
                "article": text,
                "summary": summary,
                "category": category,
                "url": url
            })
        print(news_count)
        news_count += 1

/tmp/ipython-input-3720230231.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  headline = row[0]
/tmp/ipython-input-3720230231.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = row[1]
/tmp/ipython-input-3720230231.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  date = row[2]
/tmp/ipython-input-3720230231.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer ke

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40


### Save Google_form_filling_content To CSV file


In [ ]:
import pandas as pd

# Convert the list to a pandas DataFrame
df_google_form = pd.DataFrame(google_form_filling_content)

# Save to CSV
df_google_form.to_csv("/content/drive/MyDrive/Colab Notebooks/Intern Work/google_form_data_2", index=False)

In [ ]:
DF_1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Intern Work/Google_form_data")
print(len(DF_1))

40


### Code for google Form Filling


In [ ]:
# Set up Chrome options and driver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('-incognito')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=chrome_options
)

driver.get('https://docs.google.com/forms/d/1faI49i5yTerqrhNRoVljvybsJduNMUAxsjbsGzhitTE/preview')
newspaper= 'The Tribune'
name = 'Ashish Chhipa'
count = 1
google_form_filling_content = DF_1
for i in range(len(google_form_filling_content)):
    headline = google_form_filling_content['headline'][i]
    summary = google_form_filling_content['summary'][i]
    category = google_form_filling_content['article'][i]
    date = google_form_filling_content['date'][i]
    content = google_form_filling_content['article'][i]
    url = google_form_filling_content['url'][i]
    name_field = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[1]/div/div/div[2]/div/div[1]/div/div[1]/input'))
    )
    name_field.send_keys(name)

    url_fileld = WebDriverWait(driver,5 ).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[4]/div/div/div[2]/div/div[1]/div/div[1]/input'))
    )
    url_fileld.send_keys(url)

    newspaper_field = WebDriverWait(driver,5 ).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[2]/div/div/div[2]/div/div[1]/div/div[1]/input'))
    )
    newspaper_field.send_keys(newspaper)


    date_field = WebDriverWait(driver,5 ).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[3]/div/div/div[2]/div/div/div[2]/div[1]/div/div[1]/input'))
      )
    date_field.send_keys(date)


    headline_field = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[5]/div/div/div[2]/div/div[1]/div/div[1]/input'))
      )
    headline_field.send_keys(headline)


    content_field = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[6]/div/div/div[2]/div/div[1]/div[2]/textarea'))
      )
    content_field.send_keys(content)


    summary_field = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[7]/div/div/div[2]/div/div[1]/div[2]/textarea'))
      )
    summary_field.send_keys(summary)


    category_xpath='//*[@id="i37"]/span[1]/b'
    if category == 'Politics':
          category_xpath = '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[8]/div/div/div[2]/div/div/span/div/div[4]/label/div/div[2]/div/span'
    elif category == 'International News':
          category_xpath = '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[8]/div/div/div[2]/div/div/span/div/div[1]/label/div/div[2]/div/span'
    elif category == 'National News':
          category_xpath = '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[8]/div/div/div[2]/div/div/span/div/div[2]/label/div/div[2]/div/span'
    elif category == 'Local News':
          category_xpath = '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[8]/div/div/div[2]/div/div/span/div/div[3]/label/div/div[2]/div/span'
    elif category == 'Business and Finance':
          category_xpath = '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[8]/div/div/div[2]/div/div/span/div/div[5]/label/div/div[2]/div/span'
    elif category == 'Science and Technology':
          category_xpath = '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[8]/div/div/div[2]/div/div/span/div/div[6]/label/div/div[2]/div/span'
    elif category == 'Health and Wellness':
          category_xpath = '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[8]/div/div/div[2]/div/div/span/div/div[7]/label/div/div[2]/div/span'
    elif category == 'Entertainment':
          category_xpath = '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[8]/div/div/div[2]/div/div/span/div/div[8]/label/div/div[2]/div/span'
    elif category == 'Sports':
          category_xpath = '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[8]/div/div/div[2]/div/div/span/div/div[9]/label/div/div[2]/div/span'
    elif category == 'Lifestyle and Features':
          category_xpath = '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[8]/div/div/div[2]/div/div/span/div/div[10]/label/div/div[2]/div/span'
    elif category == 'Opinion and Editorial':
          category_xpath = '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[8]/div/div/div[2]/div/div/span/div/div[11]/label/div/div[2]/div/span'
    elif category == 'Environment':
          category_xpath = '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[8]/div/div/div[2]/div/div/span/div/div[12]/label/div/div[2]/div/span'
    elif category == 'Education':
          category_xpath = '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[8]/div/div/div[2]/div/div/span/div/div[13]/label/div/div[2]/div/span'
    elif category == 'Crime and Justice':
          category_xpath = '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[8]/div/div/div[2]/div/div/span/div/div[14]/label/div/div[2]/div/span'
    elif category == 'Human Interest':
          category_xpath = '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[8]/div/div/div[2]/div/div/span/div/div[15]/label/div/div[2]/div/span'
    elif category == 'Obituaries':
          category_xpath = '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[8]/div/div/div[2]/div/div/span/div/div[16]/label/div/div[2]/div/span'
    elif category == 'Weather':
          category_xpath = '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[8]/div/div/div[2]/div/div/span/div/div[17]/label/div/div[2]/div/span'
    elif category == 'Religion and Spirituality':
          category_xpath = '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[8]/div/div/div[2]/div/div/span/div/div[18]/label/div/div[2]/div/span'
    elif category == 'Technology and Gadgets':
          category_xpath = '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[8]/div/div/div[2]/div/div/span/div/div[19]/label/div/div[2]/div/span'
    elif category == 'Automotive':
          category_xpath = '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[8]/div/div/div[2]/div/div/span/div/div[20]/label/div/div[2]/div/span'

    category_field = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH,category_xpath))
      )
    category_field.click()



    submit_field = WebDriverWait(driver, 5).until(
      EC.presence_of_element_located((By.XPATH, '//*[@id="mG61Hd"]/div[2]/div/div[3]/div[1]/div[1]/div/span/span'))
      )
    submit_field.click()


    confirmation_field = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[2]/div[1]/div/div[3]"))
     )
    if(confirmation_field.text=="Your response has been recorded."):
          print("done successfully")
          print(headline)
          print(summary)
          print(category)
          submit_another_response_link = WebDriverWait(driver, 10).until(
          EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[2]/div[1]/div/div[4]/a'))
          )
          submit_another_response_link.click()
          count+=1
          print(count)
    else:
        print("unsuccessfull")